# Notebook to scrape dynamic webpage from Copenhagen Municipality
This notebook contains the code used to scrape information from exercise facilities, collected and published by Copenhagen Municipality.
Since the table displayed on the website is generated according to filters from an interactive map, it was apparently required to use Selenium to appropriately access the desired data.

In [1]:
# import requests 
# import re 
# from bs4 import BeautifulSoup
# import pandas as pd
# import regex as re
# import os
# import time
# from selenium.webdriver import Chrome
# from selenium.webdriver import ChromeOptions
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
# from tqdm import tqdm
# import numpy as np

import imports as i
import importlib
importlib.reload(i)

c:\Users\Veronii\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'imports' from 'c:\\Users\\Veronii\\Desktop\\data-wild-west\\code\\imports.py'>

In [2]:
# Define URL
url = 'https://kbhkort.kk.dk/spatialmap?page=widget-view&name=motion/motionslisten'

In [3]:
# Initialise a dictionary to hold the scraped data
data = {'type':[], 'activity':[], 'location':[], 'website':[], 'gender':[], 'age':[], 'special':[], 'address':[]}

options = i.ChromeOptions()       # Get Chrome options
options.headless = True         # This stops an actual browser from being open and shown
driver = i.Chrome('libraries/chromedriver-win64/chromedriver.exe', options=options)  # Optional argument, if not specified will search path.
driver.get(url)

print("Driver and URL passed. Wait a second...")
driver.implicitly_wait(20) # Wait for the website to load

count = len(driver.find_elements(By.XPATH, '/html/body/div/div[4]/div/ul/li'))
print(f'There is a total of {count} entries.')

# At the time of writing, there were 606 entries (2023-11-16)
for j in i.tqdm(range(1, count+1)): 

    # Activity type/category - this is indicated by the associated icon. The icon name is collected.
    icon = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[1]/img')
    data['type'].append(icon.get_attribute('src'))

    # Name/title of the activity
    type = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[1]/strong[1]')
    data['activity'].append(type.text)

    # Location of activity - not all entries list a location
    try:
        location = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[1]/strong[2]')
        data['location'].append(location.text)
    except i.NoSuchElementException:
        data['location'].append(None)

    # Website
    try:
        site = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[1]/a')
        data['website'].append(site.get_attribute('href'))
    except i.NoSuchElementException:
        data['website'].append(None)
    

    # Gender
    gender = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[4]') # This works
    data['gender'].append(gender.text)

    # Age Group
    age = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[5]')
    data['age'].append(age.text)

    """ # Special case + address
    try:
        special = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[6]')
        if not special.text.startswith("| Særlig"):
            data['special'].append(None)
        else:
            data['special'].append(special.text)
    except NoSuchElementException:
        data['special'].append(None) """

    # Address of activity - some entries have an extra field, so this messess with the current XPath implementation
    # Therefore, each element needs to be checked that it is indeed an address.
    # Luckily, all address entries in the table contain the prefix 'Mødested' (meeting place).
    try:
        address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[6]')
        if not address.text.startswith('Mødested: ') and address.text.startswith("| Særlig"): # Check that the text is indeed the address
            data['special'].append(address.text)
            try:
                address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{j}]/div[2]/div[2]/span[7]')
                data['address'].append(address.text) #.removeprefix('Mødested: '))
            except i.NoSuchElementException:
                data['address'].append(None)
        else:
            data['special'].append(None)
            data['address'].append(address.text) #.removeprefix('Mødested: '))
    except i.NoSuchElementException:
        data['special'].append(None)
        data['address'].append(None)

driver.quit()

C:\Users\Veronii\AppData\Local\Temp\ipykernel_23272\3932252778.py:5: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True         # This stops an actual browser from being open and shown


TypeError: WebDriver.__init__() got multiple values for argument 'options'

In [4]:
# Create and inspect the dataframe
df_raw = i.pd.DataFrame.from_dict(data)
df_raw.head()

,type,activity,location,website,gender,age,special,address


In [5]:
# Save the dataframe. The encoding 'utf-16' had to be used to ensure the danish special characters of 'æ, ø, å' were present.
# Using the default encoding did not work.

#cwd = os.getcwd()
PATH = '../data/raw_data/'
path = PATH + "/kbh_facilities_v2.csv"
df_raw.to_csv(path, encoding='utf-16', index=False)

# Read in existing CSV:

In [6]:
# Read in CSV, ensure correct encoding is used.

df = i.pd.read_csv('../data/raw_data/kbh_facilities_v2.csv', encoding='utf-16').fillna(value='None')

print(f'The {len(df)} entries consist of {len(i.np.unique(df["activity"], return_counts = True)[0])} unique activity names.')
df.head()

The 0 entries consist of 0 unique activity names.


,type,activity,location,website,gender,age,special,address


# Data Cleaning
Unfortunately, there are too many unique exercise types present in the default description.

In [7]:
df_test = df.copy()

In [8]:
dict_type = {'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/boldspil_26x26.png': 'ball_sports',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/dans_26x26.png': 'dance',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/fitness_26x26.png': 'fitness',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/gym_26x26.png': 'gym',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/kampsport_26x26.png': 'martial_arts',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/loeb_26x26.png': 'running',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/natur_26x26.png': 'nature',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/svoemning_26x26.png': 'swimming',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/udemotion_26x26.png': 'outdoors',
             'https://kbhkort.kk.dk/images/ikoner/suf/sundhed/yoga_26x26.png': 'yoga'}
dict_gender = {'Køn: Begge':'both', 'Køn: Kvinder':'women', 'Køn: Mænd':'men'}
dict_age = {'| Aldersgruppe: Alle': 'all', '| Aldersgruppe: Seniorer': 'seniors'}
dict_special = {'+ 65 år':'65+',
                '+60':'60+', 
                '+60 år':'60+', 
                '+65':'65+', 
                '+65 år':'65+',
                '65+ år':'65+',
                'Kvinder 45 +':'45+', 
                'Kvinder 65+ år':'65+',
                'Mænd 65 år+':'65+',
                'PAN har fokus på inklusion af mennesker med et særligt fokus på seksuel mangfoldighed og kønsdiversitet.':'PAN har fokus på inklusion af mennesker med et særligt fokus på seksuel mangfoldighed og kønsdiversitet',
                'mænd +65 år':'65+'}
dict_address = {"":"None"}

df_test["special"] = df_test["special"].apply(lambda s:s.replace('| Særlig målgruppe: ', ''))
df_test["address"] = (df_test["address"].str.removeprefix("Mødested:")).str.strip()

mask = df_test["location"]=='Kommunal park'
df_test.loc[mask, ["activity", "location"]] = (df_test.loc[mask, ["location","activity"]].values)

In [9]:
df_test = df_test.replace({"type": dict_type, "gender": dict_gender, "age": dict_age, "special":dict_special, "address":dict_address})
df_test.tail()

,type,activity,location,website,gender,age,special,address


In [10]:
temp = {'health':['fysio', 'hjært', 'nær', 'puls', 'hjert', 'mind', 'knæ', 'ryg', 'senior', 'stabil', 'mobil'],
        'sport':['bold', 'ball', 'tennis', 'minton', 'golf', 'cricket', 'volley'], 
        'fitness':['yoga', 'træn', 'gym', 'motion', 'fitness', 'cyk', 'løb', 'stav', 'cross', 'ro', 'kajak', 'zumba', 'pilates', 'kamp', 'svøm', 'spin', 'kondi'], 
        'recreation':['billiard', 'billard', 'dart', 'dans', 'dance', 'bowl', 'gå', 'walk', 'spil', 'petanque', 'park', 'have'] 
        }

dict_activity = {}
for k,v in temp.items():
    for x in v:
        dict_activity.setdefault(x,k)

#dict_activity

temp = df_test["activity"].copy()
for j in range(len(temp)):
    for key in dict_activity.keys():
        if i.re.search(key, temp[j].lower()): #      key in temp[i].lower():
            #print(f'we replace {temp[i]} with {dict_activity[key]}')
            temp[j] = str(dict_activity[key].strip(""))
            break
        #else:
            #print(f'i is {i} for {temp[i]}, search for {key}')
    if temp[j] not in dict_activity.values():
        #print(i, temp[i])
        temp[j] = "other"

sorted(list(zip(i.np.unique(temp, return_counts=True)[0], i.np.unique(temp, return_counts=True)[1])), key = lambda x:x[1]), len(i.np.unique(temp, return_counts=True)[0])

df_test["category"] = temp


In [11]:
df_test = df_test[['type', 'activity', 'category', 'location', 'website', 'gender', 'age', 'special','address']]
df_test.tail(10)

,type,activity,category,location,website,gender,age,special,address


In [12]:
""" np.unique(df_test["gender"], return_counts = True)
np.unique(df_test["age"], return_counts = True)
len(np.unique(df_test["activity"], return_counts = True)[0]), np.unique(df_test["activity"], return_counts = True)[0] """

' np.unique(df_test["gender"], return_counts = True)\nnp.unique(df_test["age"], return_counts = True)\nlen(np.unique(df_test["activity"], return_counts = True)[0]), np.unique(df_test["activity"], return_counts = True)[0] '